<a href="https://colab.research.google.com/github/RafaelGirardin/ModeloPredicaoFazenda/blob/master/NovoModeloPredicaoFazenda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import statistics as st
import math as mt
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")

def encontrar_data_anterior_mais_proxima(lista_datas_str, data_referencia):
  datas = []
  for data_str in lista_datas_str:
    try:
      datas.append(datetime.strptime(data_str,  "%Y-%m-%d %H:%M:%S.%f"))  # Ajustar o formato se necessário
    except ValueError:
      print(f"Data inválida: {data_str}")

  datas_anteriores = [data for data in datas if data <= data_referencia]

  if datas_anteriores:
    return max(datas_anteriores).strftime( "%Y-%m-%d %H:%M:%S.%f")[:-3]  # Retornar a data como string
  else:
    return None

def encontrar_data_posterior_mais_proxima(lista_datas_str, data_referencia):
  datas = []
  for data_str in lista_datas_str:
    try:
      datas.append(datetime.strptime(data_str, "%Y-%m-%d %H:%M:%S.%f"))  # Ajustar o formato se necessário
    except ValueError:
      print(f"Data inválida: {data_str}")

  datas_posteriores = [data for data in datas if data >= data_referencia]

  if datas_posteriores:
    return min(datas_posteriores).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Retornar a data como string
  else:
    return None


dia = 0
mes = 0
dias = []

meuDicionario = []
listaQtdAlocacoes = []
coeficientesVariacao = []
colunas = ['RN','CLIENTE', 'CD_ALOCACAO', 'DATA_HORA_VAZAO', 'TURNO', 'DATA_SAIDA_CAMINHAO','DATA_CHEGADA_CAMINHAO','DESCRICAO_TIPO_CARGA','PESO_PADRAO','DISTANCIA','VAZAO_FRENTE','CLASSIFICACAO']

# DataFrame inicial
datasetNovo = pd.DataFrame(columns=colunas)
datasetNovissimo = pd.DataFrame(columns=colunas)
for i in range(4,12,1):
  pularDia = False
  for j in range(1,32,1):
    pularDia = False
    if j == 31 and i == 11:
      break
    flag = False
    listaMinutosEntreAlocacoes = []
    if ((i == 4 or i == 6 or i == 9 or i == 11) and (j == 31)):
      continue
    dia = str(j)
    if j < 10:
      dia = "0" + str(j)
      mes = str(i)
    if (i < 10):
      mes = "0" + str(i)
    url = f"https://raw.githubusercontent.com/RafaelGirardin/ModeloPredicaoFazenda/refs/heads/master/CERRADINHO/{i}/{mes}-{dia}.csv"
    dataset = pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')
    dataset['CLASSIFICACAO'] = "Erro"
    dataset['TEMPO_INFERIOR'] = 0
    dataset['MARGEM_INFERIOR'] = 0
    dataset['MARGEM_SUPERIOR'] = 0
    listaQtdAlocacoes.append(len(dataset['CD_ALOCACAO']))
    if len(dataset['CD_ALOCACAO']) == 0:
      continue
    percVazio = len(dataset.query('VAZAO_FRENTE.isnull()', engine='python'))/len(dataset['VAZAO_FRENTE'])
    if percVazio >= 0.20:
      continue

    datasetDoDiaComVazoesPreenchidas = dataset.query('(not VAZAO_FRENTE.isnull()) and VAZAO_FRENTE > 0', engine='python')
    flagPularDia = False
    for vazao in range(0, len(dataset['VAZAO_FRENTE'])):
      if np.isnan(dataset['VAZAO_FRENTE'][vazao]):
        distancia = dataset['DISTANCIA'][vazao]
        datasetDoDiaComVazoesPreenchidasParaUmaFazenda = datasetDoDiaComVazoesPreenchidas.loc[(dataset['DISTANCIA'] == distancia)]
        datetimeDataVazaoNula = datetime.strptime(dataset['DATA_HORA_VAZAO'][vazao],"%Y-%m-%d %H:%M:%S.%f")
        stringDataVazaoNaoNulaAnteriorMaisProxima = encontrar_data_anterior_mais_proxima(datasetDoDiaComVazoesPreenchidasParaUmaFazenda['DATA_HORA_VAZAO'], datetimeDataVazaoNula)
        stringDataVazaoNaoNulaPosteriorMaisProxima = encontrar_data_posterior_mais_proxima(datasetDoDiaComVazoesPreenchidasParaUmaFazenda['DATA_HORA_VAZAO'], datetimeDataVazaoNula)
        if stringDataVazaoNaoNulaAnteriorMaisProxima is None and stringDataVazaoNaoNulaPosteriorMaisProxima is None:
          if len(datasetDoDiaComVazoesPreenchidas) == 0:
            flagPularDia = True
            break
          somaVazoes = 0
          for vazaoNaoNula in range(0, len(datasetDoDiaComVazoesPreenchidas)):
            somaVazoes += datasetDoDiaComVazoesPreenchidas.iloc[vazaoNaoNula]['VAZAO_FRENTE']
          mediaVazoesDiferentesFazendas = somaVazoes/len(datasetDoDiaComVazoesPreenchidas)
          dataset['VAZAO_FRENTE'][vazao] = mediaVazoesDiferentesFazendas
          continue
        elif stringDataVazaoNaoNulaAnteriorMaisProxima is None:
          stringDataVazaoNaoNulaAnteriorMaisProxima = stringDataVazaoNaoNulaPosteriorMaisProxima
        elif stringDataVazaoNaoNulaPosteriorMaisProxima is None:
          stringDataVazaoNaoNulaPosteriorMaisProxima = stringDataVazaoNaoNulaAnteriorMaisProxima
        linhasComVazoesNaoNulasAnterioresMaisProximas = dataset.loc[dataset['DATA_HORA_VAZAO'] == stringDataVazaoNaoNulaAnteriorMaisProxima].query('(not VAZAO_FRENTE.isnull()) and VAZAO_FRENTE > 0', engine='python')
        linhasComVazoesNaoNulasPosterioresMaisProximas = dataset.loc[dataset['DATA_HORA_VAZAO'] == stringDataVazaoNaoNulaPosteriorMaisProxima].query('(not VAZAO_FRENTE.isnull()) and VAZAO_FRENTE > 0', engine='python')
        vazaoNaoNulaAnteriorMaisProxima = list(linhasComVazoesNaoNulasAnterioresMaisProximas['VAZAO_FRENTE'])[0]
        vazaoNaoNulaPosteriorMaisProxima = list(linhasComVazoesNaoNulasPosterioresMaisProximas['VAZAO_FRENTE'])[0]
        mediaVazoesNaoNulasMaisProximas = (vazaoNaoNulaAnteriorMaisProxima + vazaoNaoNulaPosteriorMaisProxima)/2
        dataset['VAZAO_FRENTE'][vazao] = mediaVazoesNaoNulasMaisProximas
    if flagPularDia:
      continue
    for alocacao in range(1, len(dataset['CD_ALOCACAO'])):
      if pd.isna(dataset['DATA_CHEGADA_CAMINHAO'][alocacao]):
        datasChegadasPreenchidas = dataset.loc[dataset['DATA_CHEGADA_CAMINHAO'].notna()]
        datasChegadasPreenchidasParaUmaFazenda = datasChegadasPreenchidas.loc[datasChegadasPreenchidas['DISTANCIA'] == dataset['DISTANCIA'][alocacao]]
        somaTempoDasViagens = 0
        mediaTempoDasViagens = 0
        if len(datasChegadasPreenchidasParaUmaFazenda) != 0:
          for data in range(0, len(datasChegadasPreenchidasParaUmaFazenda)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidasParaUmaFazenda.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidasParaUmaFazenda.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidasParaUmaFazenda)
        else:
          for data in range(0, len(datasChegadasPreenchidas)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidas.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidas.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidas)
        tempoPrevistoViagem = timedelta(minutes=mediaTempoDasViagens)
        dataset['DATA_CHEGADA_CAMINHAO'][alocacao] = (datetime.strptime(dataset['DATA_SAIDA_CAMINHAO'][alocacao],"%Y-%m-%d %H:%M:%S.%f") + tempoPrevistoViagem).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    dataset['DATA_CHEGADA_CAMINHAO'] = pd.to_datetime(dataset['DATA_CHEGADA_CAMINHAO'])
    dataset = dataset.sort_values(by='DATA_CHEGADA_CAMINHAO', ascending=True)
    dataset['DATA_CHEGADA_CAMINHAO'] = dataset['DATA_CHEGADA_CAMINHAO'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    print(dataset['DATA_CHEGADA_CAMINHAO'])
    print(dia + "-" + mes)
    print(dataset['DATA_CHEGADA_CAMINHAO'])
    amplitudeDeHorariosDaAlocacao = max([datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f") for data in dataset['DATA_SAIDA_CAMINHAO']]) - min([datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f") for data in dataset['DATA_SAIDA_CAMINHAO']])
    amplitudeEmMinutosDeHorariosDaAlocacao = float((amplitudeDeHorariosDaAlocacao.total_seconds()) / 60)
    mediaIdealTempo = amplitudeEmMinutosDeHorariosDaAlocacao/(len(dataset['DATA_SAIDA_CAMINHAO']))
    margemSuperior = amplitudeEmMinutosDeHorariosDaAlocacao/(len(dataset['DATA_SAIDA_CAMINHAO'])) + (amplitudeEmMinutosDeHorariosDaAlocacao)/(len(dataset['DATA_SAIDA_CAMINHAO']))*0.5
    margemInferior =  amplitudeEmMinutosDeHorariosDaAlocacao/(len(dataset['DATA_SAIDA_CAMINHAO'])) - (amplitudeEmMinutosDeHorariosDaAlocacao)/(len(dataset['DATA_SAIDA_CAMINHAO']))*0.5
    deltaMais = timedelta(minutes=margemSuperior)
    deltaMenos = timedelta(minutes=margemInferior)
    print(margemSuperior)
    print(margemInferior)
    for m in range(1,(len(dataset['DATA_CHEGADA_CAMINHAO'])) - 1):
      tempoInferior = pd.to_datetime(dataset['DATA_CHEGADA_CAMINHAO'][m], format='%Y-%m-%d %H:%M:%S.%f') - pd.to_datetime(dataset['DATA_CHEGADA_CAMINHAO'][m-1], format='%Y-%m-%d %H:%M:%S.%f')
      tempoSuperior = pd.to_datetime(dataset['DATA_CHEGADA_CAMINHAO'][m+1], format='%Y-%m-%d %H:%M:%S.%f') - pd.to_datetime(dataset['DATA_CHEGADA_CAMINHAO'][m], format='%Y-%m-%d %H:%M:%S.%f')
      if (tempoInferior).seconds/60 >=  margemInferior and (tempoInferior).seconds/60 <= margemSuperior:
        dataset['CLASSIFICACAO'][m] = "A"
      elif (tempoInferior).seconds/60 >=  margemInferior/1.5 and (tempoInferior).seconds/60 <= margemSuperior*1.5:
        dataset['CLASSIFICACAO'][m] = "B"
      elif (tempoInferior).seconds/60 >=  margemInferior/2.0 and (tempoInferior).seconds/60 <= margemSuperior*2.0:
        dataset['CLASSIFICACAO'][m] = "C"
      elif (tempoInferior).seconds/60 >=  margemInferior/2.5 and (tempoInferior).seconds/60 <= margemSuperior*2.5:
        dataset['CLASSIFICACAO'][m] = "D"
      elif (tempoInferior).seconds/60 >=  margemInferior/3.0 and (tempoInferior).seconds/60 <= margemSuperior*3.0:
        dataset['CLASSIFICACAO'][m] = "E"
      elif (tempoInferior).seconds/60 >=  margemInferior/3.5 and (tempoInferior).seconds/60 <= margemSuperior*3.5:
        dataset['CLASSIFICACAO'][m] = "F"
      else:
        dataset['CLASSIFICACAO'][m] = "G"
      dataset['TEMPO_INFERIOR'][m] = tempoInferior.seconds/60
      dataset['MARGEM_INFERIOR'][m] = margemInferior
      dataset['MARGEM_SUPERIOR'][m] = margemSuperior
    datasetNovo = pd.concat([datasetNovo, dataset],ignore_index=True)
  print(datasetNovo.shape)
datasetNovo = datasetNovo[datasetNovo['CLASSIFICACAO'] != 'Erro']
datasetNovo.tail(20)

0      2024-04-02 00:51:43.000000
1      2024-04-02 04:10:23.000000
2      2024-04-02 04:12:48.000000
3      2024-04-02 04:42:18.000000
4      2024-04-02 07:00:46.000000
                  ...            
161    2024-04-03 07:15:45.000000
162    2024-04-03 08:24:23.000000
163    2024-04-03 10:16:26.000000
164    2024-04-03 16:42:36.000000
165    2024-04-03 16:46:01.000000
Name: DATA_CHEGADA_CAMINHAO, Length: 168, dtype: object
02-04
0      2024-04-02 00:51:43.000000
1      2024-04-02 04:10:23.000000
2      2024-04-02 04:12:48.000000
3      2024-04-02 04:42:18.000000
4      2024-04-02 07:00:46.000000
                  ...            
161    2024-04-03 07:15:45.000000
162    2024-04-03 08:24:23.000000
163    2024-04-03 10:16:26.000000
164    2024-04-03 16:42:36.000000
165    2024-04-03 16:46:01.000000
Name: DATA_CHEGADA_CAMINHAO, Length: 168, dtype: object
12.507440476190474
4.169146825396825
0      2024-04-03 03:55:37.000000
1      2024-04-03 04:29:48.000000
2      2024-04-03 04:47:01.00

,RN,CLIENTE,CD_ALOCACAO,DATA_HORA_VAZAO,TURNO,DATA_SAIDA_CAMINHAO,DATA_CHEGADA_CAMINHAO,DESCRICAO_TIPO_CARGA,PESO_PADRAO,DISTANCIA,VAZAO_FRENTE,CLASSIFICACAO,TEMPO_INFERIOR,MARGEM_INFERIOR,MARGEM_SUPERIOR
11917,1,CERRADINHO,641766,2024-11-28 13:59:00.000,MANHA,2024-11-28 14:00:32.000,2024-11-28 20:25:52.000000,RODOTREM PROPRIO,60,74.71,120.41,A,9.750000,4.743307,14.229921
11918,1,CERRADINHO,641789,2024-11-28 16:01:00.000,MANHA,2024-11-28 16:02:42.000,2024-11-28 20:43:09.000000,RODOTREM PROPRIO,60,70.58,603.54,B,17.283333,4.743307,14.229921
11919,1,CERRADINHO,641786,2024-11-28 15:51:00.000,MANHA,2024-11-28 15:51:52.000,2024-11-28 20:48:24.000000,RODOTREM PROPRIO,60,70.58,603.54,A,5.250000,4.743307,14.229921
11920,1,CERRADINHO,641790,2024-11-28 16:11:00.000,MANHA,2024-11-28 16:11:33.000,2024-11-28 20:49:41.000000,RODOTREM PROPRIO,60,70.58,608.06,G,1.283333,4.743307,14.229921
11921,1,CERRADINHO,641792,2024-11-28 16:15:00.000,MANHA,2024-11-28 16:15:04.000,2024-11-28 20:53:47.000000,RODOTREM PROPRIO,60,70.58,578.38,B,4.100000,4.743307,14.229921
11922,1,CERRADINHO,641791,2024-11-28 16:13:00.000,MANHA,2024-11-28 16:13:57.000,2024-11-28 21:05:35.000000,RODOTREM TERCEIRO,60,70.58,578.38,A,11.800000,4.743307,14.229921
11923,1,CERRADINHO,641807,2024-11-28 18:01:00.000,MANHA,2024-11-28 18:01:20.000,2024-11-28 21:16:58.000000,RODOTREM TERCEIRO,60,74.71,62.92,A,11.383333,4.743307,14.229921
11924,1,CERRADINHO,641796,2024-11-28 16:53:00.000,MANHA,2024-11-28 16:54:43.000,2024-11-28 21:32:43.000000,RODOTREM TERCEIRO,60,70.58,516.33,B,15.750000,4.743307,14.229921
11925,1,CERRADINHO,641794,2024-11-28 16:39:00.000,MANHA,2024-11-28 16:40:59.000,2024-11-28 21:55:48.000000,RODOTREM PROPRIO,60,70.58,501.58,C,23.083333,4.743307,14.229921
11926,1,CERRADINHO,641798,2024-11-28 17:11:00.000,MANHA,2024-11-28 17:11:38.000,2024-11-28 22:02:03.000000,RODOTREM TERCEIRO,60,70.58,550.21,A,6.250000,4.743307,14.229921


In [38]:
df_shifted = datasetNovo.shift(-1)
new_df = pd.concat([datasetNovo, df_shifted], axis=1)
novos_nomes_colunas = ['RN', 'CLIENTE', 'CD_ALOCACAO', 'DATA_HORA_VAZAO', 'TURNO', 'DATA_SAIDA_CAMINHAO', 'DATA_CHEGADA_CAMINHAO', 'DESCRICAO_TIPO_CARGA', 'PESO_PADRAO', 'DISTANCIA', 'VAZAO_FRENTE', 'CLASSIFICACAO', 'TEMPO_INFERIOR', 'MARGEM_INFERIOR', 'MARGEM_SUPERIOR',
                       'RN_SEGUINTE', 'CLIENTE_SEGUINTE', 'CD_ALOCACAO_SEGUINTE', 'DATA_HORA_VAZAO_SEGUINTE', 'TURNO_SEGUINTE', 'DATA_SAIDA_CAMINHAO_SEGUINTE', 'DATA_CHEGADA_CAMINHAO_SEGUINTE', 'DESCRICAO_TIPO_CARGA_SEGUINTE', 'PESO_PADRAO_SEGUINTE', 'DISTANCIA_SEGUINTE', 'VAZAO_FRENTE_SEGUINTE', 'CLASSIFICACAO_SEGUINTE', 'TEMPO_INFERIOR_SEGUINTE', 'MARGEM_INFERIOR_SEGUINTE', 'MARGEM_SUPERIOR_SEGUINTE']
new_df.columns = novos_nomes_colunas
new_df = new_df.drop('CLIENTE', axis=1)
new_df = new_df.drop('CLIENTE_SEGUINTE', axis=1)
new_df = new_df.drop('CD_ALOCACAO', axis=1)
new_df = new_df.drop('CD_ALOCACAO_SEGUINTE', axis=1)
new_df = new_df.drop('DATA_HORA_VAZAO', axis=1)
new_df = new_df.drop('DATA_HORA_VAZAO_SEGUINTE', axis=1)
new_df = new_df.drop('CLASSIFICACAO', axis=1)
new_df = new_df.drop('RN', axis=1)
new_df = new_df.drop('RN_SEGUINTE', axis=1)
new_df = new_df.drop('DESCRICAO_TIPO_CARGA', axis=1)
new_df = new_df.drop('DESCRICAO_TIPO_CARGA_SEGUINTE', axis=1)
new_df = new_df.drop('TEMPO_INFERIOR', axis=1)
new_df = new_df.drop('MARGEM_INFERIOR', axis=1)
new_df = new_df.drop('MARGEM_SUPERIOR', axis=1)
caminhaoAnterior = pd.to_datetime(new_df['DATA_SAIDA_CAMINHAO'])
caminhaoSeguinte = pd.to_datetime(new_df['DATA_SAIDA_CAMINHAO_SEGUINTE'])
new_df['DURACAO_MINUTOS'] = (caminhaoSeguinte - caminhaoAnterior).dt.total_seconds() / 60
new_df = new_df[new_df['DURACAO_MINUTOS'] >= 0]
novaOrdem = ['DATA_SAIDA_CAMINHAO', 'DATA_SAIDA_CAMINHAO_SEGUINTE', 'PESO_PADRAO', 'PESO_PADRAO_SEGUINTE', 'VAZAO_FRENTE', 'VAZAO_FRENTE_SEGUINTE', 'DISTANCIA', 'DISTANCIA_SEGUINTE', 'TURNO', 'TURNO_SEGUINTE', 'DURACAO_MINUTOS', 'DATA_CHEGADA_CAMINHAO', 'DATA_CHEGADA_CAMINHAO_SEGUINTE','CLASSIFICACAO_SEGUINTE', 'TEMPO_INFERIOR_SEGUINTE', 'MARGEM_INFERIOR_SEGUINTE', 'MARGEM_SUPERIOR_SEGUINTE']
new_df = new_df[novaOrdem]
print(new_df.shape)
new_df.tail(20)

(6318, 17)


,DATA_SAIDA_CAMINHAO,DATA_SAIDA_CAMINHAO_SEGUINTE,PESO_PADRAO,PESO_PADRAO_SEGUINTE,VAZAO_FRENTE,VAZAO_FRENTE_SEGUINTE,DISTANCIA,DISTANCIA_SEGUINTE,TURNO,TURNO_SEGUINTE,DURACAO_MINUTOS,DATA_CHEGADA_CAMINHAO,DATA_CHEGADA_CAMINHAO_SEGUINTE,CLASSIFICACAO_SEGUINTE,TEMPO_INFERIOR_SEGUINTE,MARGEM_INFERIOR_SEGUINTE,MARGEM_SUPERIOR_SEGUINTE
11898,2024-11-28 11:39:55.000,2024-11-28 17:02:43.000,60,60,276.782222,276.782222,30.26,30.26,MANHA,MANHA,322.800000,2024-11-28 18:07:28.000000,2024-11-28 18:08:48.000000,G,1.333333,4.743307,14.229921
11900,2024-11-28 11:46:05.000,2024-11-28 13:22:00.000,60,60,276.782222,169.980000,30.26,70.58,MANHA,MANHA,95.916667,2024-11-28 18:09:07.000000,2024-11-28 18:10:47.000000,E,1.666667,4.743307,14.229921
11901,2024-11-28 13:22:00.000,2024-11-28 14:14:28.000,60,60,169.980000,380.000000,70.58,70.58,MANHA,MANHA,52.466667,2024-11-28 18:10:47.000000,2024-11-28 18:23:15.000000,A,12.466667,4.743307,14.229921
11906,2024-11-28 04:47:35.000,2024-11-28 15:28:03.000,60,60,228.470000,122.250000,74.71,74.71,NOITE,MANHA,640.466667,2024-11-28 19:19:27.000000,2024-11-28 19:30:02.000000,A,10.583333,4.743307,14.229921
11909,2024-11-28 15:15:32.000,2024-11-28 15:28:40.000,60,60,666.130000,670.930000,70.58,70.58,MANHA,MANHA,13.133333,2024-11-28 19:32:25.000000,2024-11-28 19:32:35.000000,G,0.166667,4.743307,14.229921
11911,2024-11-28 01:33:49.000,2024-11-28 17:19:29.000,60,60,181.700000,537.470000,70.58,70.58,NOITE,MANHA,945.666667,2024-11-28 19:38:10.000000,2024-11-28 19:50:48.000000,A,12.633333,4.743307,14.229921
11913,2024-11-28 15:35:29.000,2024-11-28 16:46:26.000,60,60,631.560000,276.782222,70.58,30.26,MANHA,MANHA,70.950000,2024-11-28 19:51:10.000000,2024-11-28 19:55:08.000000,B,3.966667,4.743307,14.229921
11916,2024-11-28 06:17:29.000,2024-11-28 14:00:32.000,60,60,173.940000,120.410000,70.58,74.71,MANHA,MANHA,463.050000,2024-11-28 20:16:07.000000,2024-11-28 20:25:52.000000,A,9.750000,4.743307,14.229921
11917,2024-11-28 14:00:32.000,2024-11-28 16:02:42.000,60,60,120.410000,603.540000,74.71,70.58,MANHA,MANHA,122.166667,2024-11-28 20:25:52.000000,2024-11-28 20:43:09.000000,B,17.283333,4.743307,14.229921
11919,2024-11-28 15:51:52.000,2024-11-28 16:11:33.000,60,60,603.540000,608.060000,70.58,70.58,MANHA,MANHA,19.683333,2024-11-28 20:48:24.000000,2024-11-28 20:49:41.000000,G,1.283333,4.743307,14.229921
